<a href="https://colab.research.google.com/github/dfridland/NLP/blob/HW14/NLP_DF_HW14_mental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import re
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

In [ ]:
data = []
with open('./Mental/combined_dataset.json', 'r') as json_file:
    json_list = list(json_file)
for json_str in json_list:
    data.append(json.loads(json_str)["Response"])

In [ ]:
data[:2]

["If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad feelings are part of living. \xa0They are the motivation to remove ourselves from situations and relationships which do us more harm than good.Bad feelings do feel terrible. \xa0 Your feeling of worthlessness may be good in the sense of motivating you to find out that you are much better than your feelings today.",
 'Hello, and thank you fo

In [ ]:
len(data)

3512

In [ ]:
#data = data[:5000]

In [ ]:
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        # summary = re.sub(r'<.*?>', " ", summary) # Убираем никнеймы
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [ ]:
train, test = train_test_split(data, test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [ ]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 2985
Test dataset length: 527


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [ ]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/Users/evamelissatasdemir/miniforge3/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt_bash", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

/Users/evamelissatasdemir/miniforge3/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,7.583200
1000,6.678400
1500,6.459300


TrainOutput(global_step=1641, training_loss=6.8602676159139815, metrics={'train_runtime': 647.9332, 'train_samples_per_second': 10.131, 'train_steps_per_second': 2.533, 'total_flos': 428780224512000.0, 'train_loss': 6.8602676159139815, 'epoch': 3.0})

In [ ]:
trainer.save_model()

In [ ]:
tokenizer.save_pretrained('gpt_mental_bash')
model.save_pretrained('model_gpt_mental_bash')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt_mental_bash")
model1 = AutoModelForCausalLM.from_pretrained("model_gpt_mental_bash")

In [ ]:
prefix = "I am not motivated " # "Инкапсуляция, наследование, полиморфизм"

In [ ]:
def generate(prefix, gen_legth=50):

    tokens = tokenizer(prefix, return_tensors='pt')

    size = tokens['input_ids'].shape[1]
    output = model1.generate(
        **tokens,
        #end_token=end_token_id,
        do_sample=False,
        max_length=size+gen_legth,
        repetition_penalty=5.,
        temperature=0.5,
        num_beams=10,
    )

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    print(prefix + result)

In [ ]:
generate(prefix)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I am not motivated ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ�


In [ ]:
generate('What should I do not to cry?', gen_legth=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What should I do not to cry?

I don't know what you're talking about, but there's a lot of things that can go wrong in your life. You need to be able to take care of yourself and make it through the day without having to worry about anything else. It doesn't matter how much money you have or how long you've been away from home. There are so many things that could go wrong if you aren't taking care of yourself right now. The best way to deal with these problems is to


In [ ]:
def find_sentence(filename, sentence):
    with open(filename) as file:
        for line in file:
            stringA = line
    match = re.search(sentence, stringA)

    if match:
        print('Yes!')
    else:
        print('No!')

In [ ]:
find_sentence('./train_dataset.txt', 'Вставай и иди на кухню пить чай с печеньем.')
find_sentence('./test_dataset.txt', 'Вставай и иди на кухню пить чай с печеньем')
find_sentence('./train_dataset.txt', '<REal_SM[techsupport]>')
find_sentence('./train_dataset.txt', "Протестовать будем методом отключения мобильных телефонов на максимально длительный период.")
# Предыдущая фраза есть в трейновом датасете.
find_sentence('./test_dataset.txt', "Протестовать будем методом отключения мобильных телефонов на максимально длительный период.")